# Alonso, Ariel - MFE Thesis - FIX Books processing Module

In [ ]:
import pandas as pd
import simplefix as sf
import time
import numpy as np
import matplotlib.pyplot as plt
import talib

## Defines

In [ ]:
''' FIX FIELDS CONSTANTS '''

REFRESH_TYPE = '35'
FULL_REFRESH = b'W'
INCREMENTAL_REFRESH = b'X'

TICKER = '55'
AY24D = b'AY24D'
AY24C = b'AY24C'
AY24 = b'AY24'

PRICE = '270'
SIZE = '271'

NUM_MESSAGES = '268'

MESSAGE_TYPE = '269'
BID = b'0'
OFFER = b'1'

ACTION = '279'
NEW = b'0'
CHANGE = b'1'
DELETE = b'2'

POSITION = '290'

In [ ]:
# Set the path AND specific file to process
DATA_PATH = 'C:/Users/Ariel/Documents/Maestria/TESIS/FIX/Logs FIX/MD-2019-12-18-price-depth-T2.log'

BOOK_COLUMNS = ['Bid1',
                'Bid2',
                'Bid3',
                'Bid4',
                'Bid5',
                'Ask1',
                'Ask2',
                'Ask3',
                'Ask4',
                'Ask5',
                ]
ID_COLUMNS = ['Message_type', 'Ticker']
DF_COLUMNS = ID_COLUMNS + BOOK_COLUMNS

## FIX Data loading

In [ ]:
raw_df = pd.read_csv(DATA_PATH,
                     names=['date', 'time', 'raw_data'],
                     delimiter=' ')

raw_df['Datetime'] = pd.to_datetime(raw_df['date'] + ' ' + raw_df['time'])
raw_df = raw_df.drop(['date', 'time'], axis=1)

## FIX Messages Parsing & Processing

In [ ]:
# Methods for processing the FIX messages

def refresh_type(message):
    ''' Returns the type of market data refresh type Fix message'''
    if message.get(REFRESH_TYPE) == FULL_REFRESH:
        return 'FULL'
    elif message.get(REFRESH_TYPE) == INCREMENTAL_REFRESH:
        return 'INCREMENTAL'
    else:
        print('Unknown refresh message type')


def get_message_from_df(df):
    ''' Returns a Fix Message object from the df raw data'''
    parser = sf.parser.FixParser()
    parser.append_buffer(df.raw_data)
    return parser.get_message()


def parse_full_refresh_message(message):
    '''
    Parses full refresh market data messages and returns a dict
    with actual order book data
    '''
    data = [message.get(REFRESH_TYPE).decode('utf-8'),
            message.get(TICKER).decode('utf-8')]
    empty_book_entries = 0
    for num_entry in range(1, 6):
        if message.get(MESSAGE_TYPE, num_entry) == BID:
            data.append((float(message.get(PRICE, num_entry)),
                         float(message.get(SIZE, num_entry))))
        else:
            empty_book_entries += 1
            data.append((0,0)) # empty bids in the book
    for num_entry in range(6, 11):
        if message.get(MESSAGE_TYPE, num_entry - empty_book_entries) == OFFER:
            data.append((float(message.get(PRICE, num_entry - empty_book_entries)),
                         float(message.get(SIZE, num_entry - empty_book_entries))))
        else:
            data.append((0,0)) # empty offers in the book
    return dict(zip(DF_COLUMNS, data))


def parse_incremental_refresh_message(message):
    '''
    Parses incremental refresh market data messages and returns a dict
    containing info about the incremental message information
    '''
    data_dict = {}
    data_dict['Message_type'] = message.get(REFRESH_TYPE).decode('utf-8')
    data_dict['Ticker'] = message.get(TICKER).decode('utf-8')
    for num_updates in range(1, int(message.get(NUM_MESSAGES)) + 1):
        if message.get(MESSAGE_TYPE, num_updates) == BID:
            if message.get(ACTION, num_updates) == NEW:
                data_dict['Action{}'.format(num_updates)] = ('Bid',
                                                             'New',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
            elif message.get(ACTION, num_updates) == CHANGE:
                data_dict['Action{}'.format(num_updates)] = ('Bid',
                                                             'Change',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
            elif message.get(ACTION, num_updates) == DELETE:
                data_dict['Action{}'.format(num_updates)] = ('Bid',
                                                             'Delete',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
        elif message.get(MESSAGE_TYPE, num_updates) == OFFER:
            if message.get(ACTION, num_updates) == NEW:
                data_dict['Action{}'.format(num_updates)] = ('Ask',
                                                             'New',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
            elif message.get(ACTION, num_updates) == CHANGE:
                data_dict['Action{}'.format(num_updates)] = ('Ask',
                                                             'Change',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
            elif message.get(ACTION, num_updates) == DELETE:
                data_dict['Action{}'.format(num_updates)] = ('Ask',
                                                             'Delete',
                                                             float(message.get(PRICE, num_updates)),
                                                             float(message.get(SIZE, num_updates)),
                                                             int(message.get(POSITION, num_updates)),
                                                             )
    return data_dict


def parse_fix_raw_data(df):
    ''' Parses a generic Fix message and returns a dict'''
    message = get_message_from_df(df)
    if refresh_type(message) == 'FULL':  # Book full refresh
        full_refresh_data = parse_full_refresh_message(message)
        return full_refresh_data
    elif refresh_type(message) == 'INCREMENTAL':  # Book incremental refresh
        incremental_refresh_data = parse_incremental_refresh_message(message)
        return incremental_refresh_data
    else:
        print('Unknown message type')
        return {}

In [ ]:
#  We now pass the dataframe to the parsing method
aux_df = raw_df.apply(parse_fix_raw_data, axis=1, result_type='expand')

#  Here we combine both dataframes to keep all data together in one df
df_data = pd.concat([raw_df, aux_df], axis=1)

In [ ]:
#  Now we need to split the dataframe in 3 by subsetting: AY24D, AY24C and AY24
df_AY24D = df_data[df_data['Ticker'] == 'AY24D']
df_AY24C = df_data[df_data['Ticker'] == 'AY24C']
df_AY24 = df_data[df_data['Ticker'] == 'AY24']

In [ ]:
# At this point we have the dataframes with the full refresh fields but
# the incremental messages are not yet incorporated in the book

# First we remove all the incremental messages that don't modify
# the order book and reset the indexes
df_AY24D = df_AY24D.dropna(thresh=5)
df_AY24D.reset_index(drop=True, inplace=True)

df_AY24C = df_AY24C.dropna(thresh=5)
df_AY24C.reset_index(drop=True, inplace=True)

df_AY24 = df_AY24.dropna(thresh=5)
df_AY24.reset_index(drop=True, inplace=True)

In [ ]:
# We need to parse the incremental messages and fill the order book
# for all the rows


def new_order_book(row, last_row):
    row[BOOK_COLUMNS] = last_row
    if row['Action1'][1] == 'Change':
        position = row['Action1'][0] + str(row['Action1'][4])
        row[position] = (row['Action1'][2], row['Action1'][3])
    elif row['Action1'][1] == 'Delete':
        for pos in range(row['Action1'][4], 5):
            aux = pos + 1
            position_original = row['Action1'][0] + str(aux)
            position_destination = row['Action1'][0] + str(pos)
            row[position_destination] = row[position_original]
        aux2 = row['Action1'][0] + '5'
        if not pd.isnull(row['Action2']):
            row[aux2] = (row['Action2'][2], row['Action2'][3])
    elif row['Action1'][1] == 'New':
        for pos in range(5, row['Action1'][4], -1):
            aux = pos - 1
            position_original = row['Action1'][0] + str(aux)
            position_destination = row['Action1'][0] + str(pos)
            row[position_destination] = row[position_original]
        aux2 = row['Action1'][0] + str(row['Action1'][4])
        row[aux2] = (row['Action1'][2], row['Action1'][3])
    return row[BOOK_COLUMNS]


def process_incremental_messages(df):
    ''' Incorporates the incremental messages info to the df order book'''
    for index, row in df.iterrows():
        if row['Message_type'] == 'X':
            last_row_data = df.loc[index - 1, BOOK_COLUMNS]
            new_data = new_order_book(row, last_row_data)
            df.at[index, BOOK_COLUMNS] = new_data.values


# Now we apply the processing of the incremental messages to the 3 dataframes
process_incremental_messages(df_AY24D)
process_incremental_messages(df_AY24C)
process_incremental_messages(df_AY24)

In [ ]:
# Now we want to modify these dataframes to separate the Bid and Ask tuples
# We create one new column for price and size for each Bid and Ask


def separate_bid_and_ask(df):
    ''' Takes a dataframe with a tuple of price and size and returns
    a dataframe with that data separated in two different columns'''
    for column in BOOK_COLUMNS:
        df[[column+'_Price', column+'_Size']] = pd.DataFrame(df[column].tolist(), index=df.index)
    df.drop(BOOK_COLUMNS, axis=1)


separate_bid_and_ask(df_AY24D)
separate_bid_and_ask(df_AY24C)
separate_bid_and_ask(df_AY24)

In [ ]:
'''Set datetime column as index. df must have a valid 'Datetime' column'''
df_AY24D = df_AY24D.set_index('Datetime',drop=False)
df_AY24C = df_AY24C.set_index('Datetime',drop=False)
df_AY24 = df_AY24.set_index('Datetime',drop=False)


# We now remove the duplicate indexes keeping only the last state (this caused problems)
df_AY24D = df_AY24D[~df_AY24D.index.duplicated(keep='last')]
df_AY24C = df_AY24C[~df_AY24C.index.duplicated(keep='last')]
df_AY24 = df_AY24[~df_AY24.index.duplicated(keep='last')]

## Book saving as .csv

In [ ]:
# Definition of columns to save in the .csv files
USEFUL_COLUMNS = ['Bid1_Price',
                'Bid2_Price',
                'Bid3_Price',
                'Bid4_Price',
                'Bid5_Price',
                'Ask1_Price',
                'Ask2_Price',
                'Ask3_Price',
                'Ask4_Price',
                'Ask5_Price',
                'Bid1_Size',
                'Bid2_Size',
                'Bid3_Size',
                'Bid4_Size',
                'Bid5_Size',
                'Ask1_Size',
                'Ask2_Size',
                'Ask3_Size',
                'Ask4_Size',
                'Ask5_Size',
                ]

In [ ]:
# Actual saving of the files
df_AY24D[USEFUL_COLUMNS].to_csv('C:/Users/Ariel/Documents/Maestria/TESIS/Mi Tesis/AY24D_18-12-19.csv')
df_AY24C[USEFUL_COLUMNS].to_csv('C:/Users/Ariel/Documents/Maestria/TESIS/Mi Tesis/AY24C_18-12-19.csv')
df_AY24[USEFUL_COLUMNS].to_csv('C:/Users/Ariel/Documents/Maestria/TESIS/Mi Tesis/AY24_18-12-19.csv')